In [1]:
import os
import pandas as pd
from athina.evals import ContextContainsEnoughInformation
from athina.loaders import Loader
from athina.keys import AthinaApiKey, OpenAiApiKey
from athina.interfaces.athina import AthinaExperiment
from athina.datasets import yc_query_mini

/Users/akshat_g/athina/repos/athina-evals/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/akshat_g/athina/repos/athina-evals/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Configure your API keys

Evals use OpenAI, so you need to configure your OpenAI API key.

If you wish to view the results on Athina's UI, and maintain a historical record of experiments, then you also need an Athina API Key.

In [2]:
from dotenv import load_dotenv

load_dotenv()

OpenAiApiKey.set_key(os.getenv('OPENAI_API_KEY'))
AthinaApiKey.set_key(os.getenv('ATHINA_API_KEY')) # Optional, recommended

### Load your dataset

You can use one of our `loaders` to load the data from a Dictionary, CSV or JSON file.

Here's an example
```
from athina.loaders import Loader

dataset = Loader().load_dict(raw_data)
```

Here is the complete [documentation](https://docs.athina.ai/evals/running_evals/loading_data) specifying the various ways you can load your dataset.

In [3]:
# Create or load batch dataset
raw_data = yc_query_mini.data
dataset = Loader().load_dict(raw_data)

pd.DataFrame(dataset)

,query,context,response,expected_response
0,What are some successful companies that went t...,[Y Combinator has invested in companies in var...,"Airbnb, Dropbox, Stripe, Reddit, Coinbase, Ins...",Airbnb and Stripe are 2 of the successful comp...
1,In which city is YC located?,"[Y Combinator is located in Mountain View, Cal...",Y Combinator is located in San Francisco,"YC is located in Mountain View, California."
2,How much equity does YC take?,[Y Combinator invests $500k in 200 startups tw...,YC invests $150k for 7%.,I cannot answer this question as I do not have...
3,How much equity does YC take?,[Y Combinator invests $500k in 200 startups tw...,I cannot answer this question as I do not have...,I cannot answer this question as I do not have...
4,Who founded YC and when was it founded?,[Y Combinator was founded in March 2005 by Pau...,Y Combinator was founded in 2005,Y Combinator was founded in March 2005 by Paul...
5,Does Y Combinator invest in startups outside t...,[Y Combinator invests in startups from all ove...,"Yes, Y Combinator invests in international sta...","Yes, Y Combinator invests in startups from all..."
6,How much does YC invest in startups?,[YC invests $150k for 7%.],$150k,YC invests $150k for 7%.
7,What is YC's motto?,[Y Combinator's motto is 'Make something peopl...,Make something people want,Make something people want


### Describe your experiment metadata fields (optional)
These metadata fields are only used as identifiers when we save your experiment on Athina Develop.
This helps you search, sort and filter through past experimentation runs.

Currently, this includes your:
- `experiment_name`: (string) The name of your experiment
- `experiment_description`: (string) A description this iteration of your experiment
- `language_model_provider`: (string) `openai`
- `language_model_id`: (string) The language model used for the LLM inference (ex: `gpt-3.5-turbo`)
- `prompt_template`: (object) A JS object representing the prompt you are sending to the LLM (for example, messages array in OpenAI)
- `dataset_name`: (string) An identifier for the dataset you are using.

In [4]:
# Define your experiment parameters
prompt_template = [
    { 
        "role": "system",
        "content": "You are an expert at answering questions about Y Combinator. If you do not know the answer, say I don't know. Be direct and concise in your responses" },
    { 
        "role": "user", 
        "content": "{query}"
    }
]
experiment = AthinaExperiment(
    experiment_name="ContextRelevance",
    experiment_description="Checking retrieval scores for YC dataset with a simple zero-shot prompt",
    language_model_provider="openai",
    language_model_id="gpt-3.5-turbo",
    prompt_template=prompt_template,
    dataset_name="yc_dataset_mini",
)

### Run your evaluation

Simply instantiate the evaluator class you wish to use, and call `run_batch` to the eval

##### Run evals in parallel (much faster)

You may specify `max_parallel_evals` to run multiple LLM evaluation inferences in parallel.

##### View as a dataframe
Call `.to_df()` on the results to view as a dataframe


##### Log results to Athina Develop (Dashboard UI)
If you have specified an `AthinaApiKey`, then results will automatically logged to the dashboard.

In [5]:
# Checks if the LLM response answers the user query sufficiently
results = ContextContainsEnoughInformation().configure_experiment(experiment).run_batch(
    data=dataset,
    max_parallel_evals=5 # Run up to 5 evals in parallel
)

results.to_df()

201


,query,context,response,expected_response,display_name,failed,grade_reason,runtime,model,passed
0,What are some successful companies that went through YC?,"[Y Combinator has invested in companies in various fields like FinTech, Healthcare, AI, etc.]","Airbnb, Dropbox, Stripe, Reddit, Coinbase, Instacart.",Airbnb and Stripe are 2 of the successful companies that went through YC.,Context Contains Enough Information,True,"The context provided does not include specific examples of successful companies that went through Y Combinator (YC). It only mentions that YC has invested in companies across various fields such as FinTech, Healthcare, AI, etc. Without specific company names or examples, the chatbot cannot answer the user's query based solely on the context given.",4005,gpt-4-1106-preview,0.0
1,In which city is YC located?,"[Y Combinator is located in Mountain View, California.]",Y Combinator is located in San Francisco,"YC is located in Mountain View, California.",Context Contains Enough Information,False,"The context clearly states that Y Combinator is located in Mountain View, California. Therefore, the chatbot can use this information to answer the user's query about the city in which YC is located, which is Mountain View.",2456,gpt-4-1106-preview,1.0
2,How much equity does YC take?,[Y Combinator invests $500k in 200 startups twice a year.],YC invests $150k for 7%.,I cannot answer this question as I do not have enough information.,Context Contains Enough Information,True,"The context provided states the amount Y Combinator invests in startups and how often, but it does not specify the percentage of equity YC takes in return for its investment. Therefore, the chatbot cannot answer the user's query about YC's equity stake based solely on the provided context.",3083,gpt-4-1106-preview,0.0
3,How much equity does YC take?,[Y Combinator invests $500k in 200 startups twice a year.],I cannot answer this question as I do not have enough information.,I cannot answer this question as I do not have enough information.,Context Contains Enough Information,True,"The context provided only states the amount Y Combinator invests in startups and how often, but it does not specify the percentage of equity YC takes in return for its investment. Therefore, the chatbot cannot answer the user's query about YC's equity take with the information given.",3361,gpt-4-1106-preview,0.0
4,Who founded YC and when was it founded?,"[Y Combinator was founded in March 2005 by Paul Graham, Jessica Livingston, Trevor Blackwell, and Robert Tappan Morris.]",Y Combinator was founded in 2005,"Y Combinator was founded in March 2005 by Paul Graham, Jessica Livingston, Trevor Blackwell, and Robert Tappan Morris.",Context Contains Enough Information,False,"The context provides the names of the founders of Y Combinator (Paul Graham, Jessica Livingston, Trevor Blackwell, and Robert Tappan Morris) and the date it was founded (March 2005), which directly answers the user's query about who founded YC and when it was founded.",2833,gpt-4-1106-preview,1.0
5,Does Y Combinator invest in startups outside the US?,[Y Combinator invests in startups from all over the world.],"Yes, Y Combinator invests in international startups as well as US startups.","Yes, Y Combinator invests in startups from all over the world.",Context Contains Enough Information,False,"The context clearly states that Y Combinator invests in startups from all over the world, which directly answers the user's query about whether Y Combinator invests in startups outside the US. Therefore, the chatbot can use the context information to affirm that Y Combinator does invest in startups outside the US.",4886,gpt-4-1106-preview,1.0
6,How much does YC invest in startups?,[YC invests $150k for 7%.],$150k,YC invests $150k for 7%.,Context Contains Enough Information,False,The context directly answers the user's query by stating that YC invests $150k for 7% equity in startups. This information is sufficient to inform the user